In [7]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
import tkinter as tk
from tkinter import messagebox, ttk


# Load the trained SVM model and scaler
svm_model_path = 'svm_investment_new.joblib'
scaler_path = 'svm_scalerr_new.joblib'

svm_model = joblib.load(svm_model_path)  # Load the trained SVM model
scaler = joblib.load(scaler_path)  # Load the scaler for consistent scaling


# Reference features expected by the model
reference_features = [
    'fuel', 'capacity', 'calculated_cost', 'irradiance_wind', 'annual_revenue',
    'annual_operating_expense', 'annual_profit', 'payback_period', 'npv', 'roi', 'irr'
]


# Prediction function with ROI, cost checks, and additional conditions
def predict_investment(user_input):
    if isinstance(user_input, dict):
        user_input = pd.DataFrame([user_input], columns=reference_features)

    # Fill missing columns with default values
    for column in reference_features:
        if column not in user_input.columns:
            user_input[column] = 0

    user_input = user_input[reference_features]  # Ensure correct order

    calculated_cost = user_input.loc[0, 'calculated_cost']
    capacity = user_input.loc[0, 'capacity']

    # Calculate derived metrics
    irradiance_wind = (1.5 * capacity * 365 * 24) / 8.67  # New formula
    annual_revenue = ((0.2 * irradiance_wind * 1000) + (365 * 0.53)) / 1_000_000  # In million pounds
    annual_operating_expense = (0.025 * irradiance_wind * 1000) / 1_000_000  # In million pounds
    annual_profit = annual_revenue - annual_operating_expense  # In million pounds

    # Calculate Net Present Value (NPV)
    npv = (annual_profit * (1 - (1 + 0.053) ** -25)) / 0.053
    user_input.loc[0, 'npv'] = npv

    # Calculate ROI
    roi = (npv / calculated_cost) * 100
    user_input.loc[0, 'roi'] = roi

    # Calculate IRR
    irr = (((npv / calculated_cost) - 1) / 25) * 100
    user_input.loc[0, 'irr'] = irr

    # Calculate Payback Period in years
    payback_period = calculated_cost / annual_profit  # Payback period in years
    user_input.loc[0, 'payback_period'] = payback_period

    # Check additional conditions
    if roi <= 100:
        return {'Investment Decision': 'Do Not Invest', 'Reason': 'ROI is lower than 100%'}

    if calculated_cost >= capacity * 2:
        return {'Investment Decision': 'Do Not Invest', 'Reason': 'Cost is too much high for this project. please enter a valid input'}

    if calculated_cost <= capacity / 2:
        return {'Investment Decision':'Do Not Invest', 'Reason': 'Cost is too much low for this project. please enter a valid input'}

    # Predict using the trained SVM model
    user_input_scaled = scaler.transform(user_input)

    prediction = svm_model.predict(user_input_scaled)

    # Convert the prediction to a human-readable label
    investment_recommended = 'Invest' if prediction[0] == 1 else 'Do Not Invest'

    # Return the prediction and key metrics
    return {
        'Investment Decision': investment_recommended,
        'ROI': f"{roi:.2f}%",
        'Payback Period (Years)': f"{payback_period:.2f}",
        'IRR': f"{irr:.2f}%",
        'NPV': f"{npv:.2f}"
    }


# Function to process user inputs and display results
def process_input():
    try:
        # Gather inputs from the GUI
        fuel = int(fuel_var.get())  # Use the variable set by radiobuttons
        capacity = int(capacity_entry.get())
        calculated_cost = int(cost_entry.get())
        irradiance_wind = int(irradiance_wind_entry.get())  # This is just for GUI, not used in calculations

        # Calculate derived metrics with the correct formula for irradiance/wind
        irradiance_wind = (1.5 * capacity * 365 * 24) / 8.67  # Derived irradiance_wind
        annual_revenue = ((0.2 * irradiance_wind * 1000) + (365 * 0.53)) / 1_000_000  # In million pounds
        annual_operating_expense = (0.025 * irradiance_wind * 1000) / 1_000_000  # In million pounds
        annual_profit = annual_revenue - annual_operating_expense  # In million pounds

        # Create the user input dictionary with derived values
        user_input_example = {
            'fuel': fuel,
            'capacity': capacity,
            'calculated_cost': calculated_cost,
            'irradiance_wind': irradiance_wind,
            'annual_revenue': annual_revenue,
            'annual_operating_expense': annual_operating_expense,
            'annual_profit': annual_profit
        }

        # Get the prediction and display it in a messagebox
        result = predict_investment(user_input_example)

        result_message = "\n".join([f"{key}: {value}" for key, value in result.items()])
        messagebox.showinfo("Investment Decision", result_message)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")


# Tkinter GUI setup
root = tk.Tk()
root.title("Investment Recommendation Predictor")
root.geometry("400x300")
root.configure(bg='lightblue')

fuel_var = tk.IntVar(value=1)  # Default to Solar (1)

tk.Label(root, text="Select Renewable Source:", bg='lightblue', fg='black').grid(row=1, column=0, columnspan=2)
tk.Radiobutton(root, text="Solar", variable=fuel_var, value=1, bg='lightblue', fg='black').grid(row=2, column=0)
tk.Radiobutton(root, text="Wind", variable=fuel_var, value=0, bg='lightblue', fg='black').grid(row=2, column=1)

tk.Label(root, text="Capacity (MW)", bg='lightblue', fg='black').grid(row=3, column=0)
capacity_entry = tk.Entry(root)
capacity_entry.grid(row=3, column=1)

tk.Label(root, text="Investment (Million £)", bg='lightblue', fg='black').grid(row=4, column=0)
cost_entry = tk.Entry(root)
cost_entry.grid(row=4, column=1)

tk.Label(root, text="Irradiance/Wind", bg='lightblue', fg='black').grid(row=5, column=0)
irradiance_wind_entry = tk.Entry(root)  # Dummy value, not used in calculations
irradiance_wind_entry.grid(row=5, column=1)

tk.Label(root, text="Select City", bg='lightblue', fg='black').grid(row=0, column=0)
city_var = tk.StringVar(value="Manchester")  # Default city
cities = ["Manchester", "Birmingham", "Sheffield", "London", "Bristol", "Leeds"]  # List of cities
city_menu = ttk.OptionMenu(root, city_var, *cities)
city_menu.grid(row=0, column=1)

predict_button = tk.Button(root, text="Calculate", command=process_input, bg='lightgreen', fg='black')
predict_button.grid(row=6, column=1)

root.mainloop()  # Start the Tkinter event loop
